In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
with open('train-en/left.txt') as fopen:
    left = fopen.read().split('\n')

In [3]:
import malaya

/home/husein/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/husein/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

In [4]:
from malaya.text.rules import rules_normalizer, rules_compound_normalizer
from malaya.text.normalization import _is_number_regex
from malaya.text.function import replace_punct
from collections import defaultdict
import re
import random
import json

In [5]:
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\]^_`{|}~'

def case_of(text):
    return (
        str.upper
        if text.isupper()
        else str.lower
        if text.islower()
        else str.title
        if text.istitle()
        else str
    )

def strip_punct(word):
    left = []
    right = []
    i = 0
    while i < len(word):
        if word[i] in PUNCTUATION:
            left.append(word[i])
            i += 1
        else:
            break
    i = len(word) - 1
    while i > 0:
        if word[i] in PUNCTUATION:
            right.append(word[i])
            i -= 1
        else:
            break
    left = ''.join(left)
    right = ''.join(right[::-1])
    if len(right):
        word_ = word[:-len(right)]
    else:
        word_ = word
    word_ = word_[len(left):]
    return left, right, word_

In [6]:
def replace_words_punct(left_word, right_word):
    left_left, left_right, left_word = strip_punct(left_word)
    right_left, right_right, right_word = strip_punct(right_word)
    return f'{left_left}{right_word}{left_right}'

def random_replace_alignment(left, right, alignment, min_replace = 5, max_replace = 15):
    splitted_left = left.split()
    splitted_right = right.split()
    
    selected_alignment = []
    for s in alignment:
        l = s[0]
        r = s[1]
        try:
            if _is_number_regex(splitted_left[l].replace(',', '').replace('.', '')) or _is_number_regex(splitted_right[r].replace(',', '').replace('.', '')):
                continue
            elif splitted_left[l].isupper() or splitted_right[r].isupper():
                continue
            elif splitted_left[l].lower() == splitted_right[r].lower():
                continue
            elif splitted_left[r].lower() in ['the', 'a', 'an', 'it', 'is', 'are']:
                continue
            else:
                selected_alignment.append((l, r))
        except:
            pass
    
    try:
        count_replace = random.randint(min_replace, min(max_replace, len(selected_alignment)))
        selected = random.sample(selected_alignment, count_replace)
        for s in selected:
            splitted_left[s[0]] = replace_words_punct(splitted_left[s[0]], splitted_right[s[1]])

        return ' '.join(splitted_left), selected
    
    except:
        return ' '.join(splitted_left), []

In [7]:
model = malaya.language_detection.substring_rules()

In [8]:
tokenizer = malaya.tokenizer.Tokenizer()

In [11]:
l = ' '.join(tokenizer.tokenize(left[0]))
predicted = model.predict(l.split())
predicted

['EN', 'EN', 'EN', 'EN', 'EN', 'EN', 'EN', 'EN', 'NOT_LANG', 'NOT_LANG']

In [12]:
l.split()

['Most', 'of', 'them', 'are', 'teenagers', 'or', 'in', 'their', '20s', '.']

In [13]:
from tqdm import tqdm

def loop(rows):
    rows, _ = rows
    strings, labels = [], []
    for i in tqdm(range(len(rows))):
        try:
            left_ = rows[i]
            tokenized_l = tokenizer.tokenize(left_)
            l = tokenized_l
            predicted = model.predict(l)
            strings.append(l)
            labels.append(predicted)
        except Exception as e:
            print(e)
    
    return [[strings, labels]]

In [14]:
r = loop((left[:10],0))

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1525.65it/s]


In [18]:
r[0][0][0]

['Most', 'of', 'them', 'are', 'teenagers', 'or', 'in', 'their', '20s', '.']

In [20]:
r[0][1][0]

['EN', 'EN', 'EN', 'EN', 'EN', 'EN', 'EN', 'EN', 'NOT_LANG', 'NOT_LANG']

In [21]:
import mp

In [23]:
r = mp.multiprocessing(left[400000:700000], loop, cores = 4)

100%|████████████████████████████████████████████████████████████████████████████████████████| 75000/75000 [00:22<00:00, 3406.53it/s]


In [24]:
strings, labels = [], []
for i in range(len(r)):
    print(i, len(r[i][0]))
    strings.extend(r[i][0])
    labels.extend(r[i][1])

0 75000
1 75000
2 75000
3 75000


In [26]:
strings[-1], labels[-1]

(['But',
  'I',
  'believe',
  'it',
  'has',
  'an',
  'assimilation',
  'problem',
  '.',
  'Muslim',
  'minorities',
  'there',
  'seem',
  'to',
  'be',
  'less',
  'integrated',
  'than',
  'in',
  'other',
  'European',
  'countries',
  '.',
  'There',
  'are',
  '.',
  'that',
  'while',
  'Muslims',
  'are',
  'around',
  '7%',
  'to',
  '8%',
  'of',
  "France's",
  'population',
  ',',
  'they',
  'are',
  'more',
  'than',
  '60%',
  'of',
  'its',
  'prison',
  'population',
  '.',
  'A',
  '.',
  'also',
  'showed',
  'that',
  'four',
  'of',
  'the',
  'five',
  'most',
  'radicalized',
  'Muslim',
  'populations',
  'in',
  'the',
  'world',
  'are',
  'in',
  'countries',
  'of',
  'French',
  'culture',
  '.',
  'One',
  'factor',
  'could',
  'be',
  'the',
  'very',
  'aggressive',
  'form',
  'of',
  'secularism',
  'in',
  'France',
  '-',
  '-',
  'far',
  'more',
  'aggressive',
  'than',
  'the',
  'United',
  'States',
  'or',
  'any',
  'other',
  'Western',


In [27]:
with open('en-substrings.json', 'w') as fopen:
    json.dump({'strings': strings, 'labels': labels}, fopen)